In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=200, n_features=6, n_informative=3, 
                          n_redundant=0, random_state=42, n_classes=2,
                          class_sep=1.8)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

svm = SVC(kernel='linear', C=10)
svm.fit(X_pca, y)

x_min, x_max = X_pca[:, 0].min() - 0.5, X_pca[:, 0].max() + 0.5
y_min, y_max = X_pca[:, 1].min() - 0.5, X_pca[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 30),
                     np.linspace(y_min, y_max, 30))

w = svm.coef_[0]
b = svm.intercept_[0]
zz = (-w[0] * xx - w[1] * yy - b) / w[2]

scatter_real = go.Scatter3d(
    x=X_pca[y == 0, 0], y=X_pca[y == 0, 1], z=X_pca[y == 0, 2],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Real Banknotes'
)

scatter_counterfeit = go.Scatter3d(
    x=X_pca[y == 1, 0], y=X_pca[y == 1, 1], z=X_pca[y == 1, 2],
    mode='markers',
    marker=dict(size=5, color='red', symbol='diamond', opacity=0.8),
    name='Counterfeit'
)
decision_plane = go.Surface(
    x=xx, y=yy, z=zz,
    colorscale='Greens',
    opacity=0.5,
    showscale=False,
    name='Decision Boundary'
)
support_vectors = go.Scatter3d(
    x=X_pca[svm.support_, 0], 
    y=X_pca[svm.support_, 1], 
    z=X_pca[svm.support_, 2],
    mode='markers',
    marker=dict(size=7, color='gold', symbol='x', opacity=1),
    name='Support Vectors'
)
layout = go.Layout(
    title='Interactive 3D SVM Classification: Swiss Banknotes',
    scene=dict(
        xaxis_title='First Principal Component',
        yaxis_title='Second Principal Component',
        zaxis_title='Third Principal Component',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=0.8)
        )
    ),
    legend=dict(orientation="h"),
    margin=dict(l=0, r=0, b=0, t=30),
    height=700
)

fig = go.Figure(data=[scatter_real, scatter_counterfeit, decision_plane, support_vectors], 
                 layout=layout)
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=["type", "surface"],
                    label="Show Decision Plane",
                    method="restyle"
                ),
                dict(
                    args=["type", "none"],
                    label="Hide Decision Plane",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.05,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"marker.size": [5, 5, 7, 7]}],
                    label="Enlarge Points",
                    method="restyle"
                ),
                dict(
                    args=[{"marker.size": [3, 3, 5, 5]}],
                    label="Default Size",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.35,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)
fig.write_html("svm_3d_rotation_plot.html", include_plotlyjs=True)